<a href="https://colab.research.google.com/github/mbCap/Fallstudie_BESS_Degradation/blob/main/Latest_von_Fallstudie_BESS_Degradation_3_Zyklen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fallstudie BESS

In [ ]:
!git clone https://github.com/AlexKressner/WS24_Supply_Chain_Optimierung

fatal: destination path 'WS24_Supply_Chain_Optimierung' already exists and is not an empty directory.


In [ ]:
!pip install -q pyscipopt

In [ ]:
import pandas as pd
from pyscipopt import Model, quicksum

# Initialisiere das Modell
scip = Model()

# Daten laden
folder = "WS24_Supply_Chain_Optimierung/Daten/Fallstudie"
preisprognose = pd.read_excel(f"{folder}/Preisprognosen.xlsx")

# Indexmenge (Stunden des Folgetages)
T = range(1, 25)

# Parameter (unverändert)
cap = 40.0  # Nominelle Kapazität in [MWh]
DoD = 0.80  # Depth of Discharge
SOC_min = cap * (1 - DoD)  # Minimaler Ladezustand
SOC_max = cap  # Maximaler Ladezustand
c_rate = 0.5  # C-Rate
eta_rte = 0.975  # Round-trip Efficiency
eta_wr = 0.985  # Wirkungsgrad Wechselrichter
cyclecost = 1500.0  # Fixkosten pro Zyklus
maxCycles = 3.0  # Maximale Zyklen pro Tag (3 Zyklen)

# Marktpreise (Durchschnitt pro Stunde)
p_Markt = preisprognose.groupby("Stunde")["Strompreis"].mean().tolist()

# Lade- und Entladeverluste
eta_charge = eta_rte * eta_wr  # Ladeverlust
eta_discharge = eta_rte * eta_wr  # Entladeverlust

# Zählvariable für verbrauchte Zyklen
Z = scip.addVar(
    vtype="CONTINUOUS",
    lb=0,
    ub=maxCycles,
    name="cycles_used"
)

# Entscheidungsvariablen: SOC, charge und discharge
SOC = {t: scip.addVar(vtype="CONTINUOUS", lb=SOC_min, ub=SOC_max, name=f"SOC_{t}") for t in T}
charge = {t: scip.addVar(vtype="CONTINUOUS", lb=0, ub=c_rate * cap, name=f"charge_{t}") for t in T}
discharge = {t: scip.addVar(vtype="CONTINUOUS", lb=0, ub=c_rate * cap, name=f"discharge_{t}") for t in T}

# Nebenbedingungen
scip.addCons(SOC[1] == 0.5 * cap)
scip.addCons(SOC[24] == 0.5 * cap)
scip.addCons(discharge[24] == 0)

# Ladezustandsdynamik
for t in T:
    if t == 1:
        continue
    scip.addCons(
        SOC[t] == SOC[t - 1] + eta_charge * charge[t - 1] - (1 / eta_discharge) * discharge[t - 1]
    )

# Zyklenbegrenzung (Berücksichtige 3 Zyklen pro Tag)
throughput = quicksum(charge[t] + discharge[t] for t in T)
scip.addCons(Z >= throughput / (2.0 * cap * DoD))
scip.addCons(Z <= maxCycles)

# Degradation: 2% pro Jahr
def calculate_degradation(year):
    degradation_factor = 1 - 0.02 * year  # 2% Degradation pro Jahr
    return degradation_factor

# Zielfunktion: Erlöse - Kosten für Strom - Zykluskosten
scip.setObjective(
    quicksum(p_Markt[t - 1] * discharge[t] - p_Markt[t - 1] * charge[t] for t in T) - cyclecost * Z,
    sense="maximize"
)

# Modell lösen
scip.optimize()

# Ausgabe der Ergebnisse
initial_revenue = scip.getObjVal()
print(f"Optimaler Zielfunktionswert (Jahr 0): {initial_revenue:.2f} € pro Tag / {initial_revenue * 365:.2f} € pro Jahr")

# Berechnung des Erlöses mit Degradation für mehrere Jahre
for year in range(1, 11):
    degradation_factor = calculate_degradation(year)
    revenue_today = scip.getObjVal()
    revenue_today *= degradation_factor  # Berücksichtige Degradation
    revenue_in_year = revenue_today * 365  # Jahreswert
    revenue_loss_per_day = initial_revenue - revenue_today
    revenue_loss_per_year = initial_revenue * 365 - revenue_in_year

    print(f"Jahr {year}: {revenue_today:.2f} € pro Tag / {revenue_in_year:.2f} € pro Jahr "
          f"(Erlösverlust pro Tag: {revenue_loss_per_day:.2f} € / Erlösverlust pro Jahr: {revenue_loss_per_year:.2f} €)")

# Berechnung der Veränderung des Erlöses
print("\nWie verändert sich der Erlös eines Tages, wenn ich eine gewisse Degradation unterstelle (2% pro Jahr für die Batterie)?")
print(f"Der Erlös eines Tages sinkt aufgrund der Degradation im 10. Jahr um {100 * (1 - calculate_degradation(10)):.2f}% im Vergleich zum initialen Jahr.")


Optimaler Zielfunktionswert (Jahr 0): 1885.12 € pro Tag / 688068.14 € pro Jahr
Jahr 1: 1847.42 € pro Tag / 674306.78 € pro Jahr (Erlösverlust pro Tag: 37.70 € / Erlösverlust pro Jahr: 13761.36 €)
Jahr 2: 1809.71 € pro Tag / 660545.42 € pro Jahr (Erlösverlust pro Tag: 75.40 € / Erlösverlust pro Jahr: 27522.73 €)
Jahr 3: 1772.01 € pro Tag / 646784.05 € pro Jahr (Erlösverlust pro Tag: 113.11 € / Erlösverlust pro Jahr: 41284.09 €)
Jahr 4: 1734.31 € pro Tag / 633022.69 € pro Jahr (Erlösverlust pro Tag: 150.81 € / Erlösverlust pro Jahr: 55045.45 €)
Jahr 5: 1696.61 € pro Tag / 619261.33 € pro Jahr (Erlösverlust pro Tag: 188.51 € / Erlösverlust pro Jahr: 68806.81 €)
Jahr 6: 1658.90 € pro Tag / 605499.97 € pro Jahr (Erlösverlust pro Tag: 226.21 € / Erlösverlust pro Jahr: 82568.18 €)
Jahr 7: 1621.20 € pro Tag / 591738.60 € pro Jahr (Erlösverlust pro Tag: 263.92 € / Erlösverlust pro Jahr: 96329.54 €)
Jahr 8: 1583.50 € pro Tag / 577977.24 € pro Jahr (Erlösverlust pro Tag: 301.62 € / Erlösverlust p